In [8]:
import numpy as np
import pandas as pd
import os

In [9]:
data_directory = "../Data/Study/11/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [10]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [11]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [12]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [13]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.makedirs(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [14]:
calculate_errors("error/recalibrated_calibration")

hallway_20230509182317_shifted0_recalibrated_calibration.csv
uncalibrated: 29.48868748537393
calibrated: 20.37724843094
cosine: 27.97946338192694

wsBodyConstrained_20230509181907_shifted0_recalibrated_calibration.csv
uncalibrated: 24.62532949562008
calibrated: 25.91709602020908
cosine: 23.746845133282413

wsWalking_20230509181508_shifted11_recalibrated_calibration.csv
uncalibrated: 62.08388215376997
calibrated: 61.89398084185732
cosine: 89.49185488702328

ssHeadConstrained_20230509181201_shifted22_recalibrated_calibration.csv
uncalibrated: 14.33698195032697
calibrated: 1.6677059614611127
cosine: 14.025689333332373

calibration_20230509181719_shifted0_recalibrated_calibration.csv
uncalibrated: 13.044310728113068
calibrated: 1.2232735227593357
cosine: 12.744002553701247

calibration_20230509183012_shifted0_recalibrated_calibration.csv
uncalibrated: 13.935105204489094
calibrated: 1.5809743163238046
cosine: 13.582494126898009

hallway_20230509180912_shifted0_recalibrated_calibration.csv
u